In [1]:
!pip install pandas==0.23.4
#.asfreq()

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
%matplotlib inline

import psycopg2
from itertools import chain
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re
import pytz
import math as math
from scipy.interpolate import interp1d
import sys


import sys
from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os
from sagemaker import get_execution_role

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
sagemaker_session = sagemaker.Session()

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [5]:
s3_bucket = 'mtgmlbucketmultseries'  # replace with an existing bucket if needed
s3_prefix = 'mtgml-notebook'    # prefix used for all data stored within the bucket

role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

In [6]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)


In [7]:
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


In [9]:
freq='D'
prediction_length=7
context_length=7

start_date_othervarbs=pd.Timestamp("2019-10-16", freq=freq)
start_date = pd.Timestamp("2019-10-23", freq=freq)
end_dataset = pd.Timestamp("2020-11-25", freq=freq)
end_training = end_dataset-timedelta(days=7)
mid_dataset=pd.Timestamp("2020-09-15",freq=freq)
end_testing = end_dataset
print(f'End training: {end_training}, End testing: {end_testing}')

End training: 2020-11-18 00:00:00, End testing: 2020-11-25 00:00:00


In [10]:
cur=conn.cursor()

In [11]:
cur.execute('''SELECT ebayname.cardname FROM 
((SELECT DISTINCT(cardname) FROM transactions) AS ebayname 
INNER JOIN 
(SELECT pio.cardname FROM ((SELECT DISTINCT(cardname) FROM tourninfo) AS pio 
INNER JOIN 
(SELECT DISTINCT(cardname) FROM moderntourninfo) AS mod ON pio.cardname=mod.cardname)) 
as overall ON ebayname.cardname=overall.cardname)''')

In [12]:
analynames=cur.fetchall()

In [13]:
len(analynames)

701

In [14]:
analynames[0]

('Marauding Raptor',)

In [ ]:
#analyname='Supreme Verdict'
#for loop over cardnames starts here
training_data=[]
test_data=[]
target_series=[]
for cat,analyname in enumerate(analynames):
    cur.execute('''SELECT carddate,price,cardtype FROM public.totalgoatbot WHERE cardname ILIKE %s''',analyname)
    print(analyname)
    goatbotrows=cur.fetchall()

    goatdf=pd.DataFrame(goatbotrows,columns=["carddate","price","cardtype"]).set_index("carddate")
    
    goatdf.index=pd.to_datetime(goatdf.index,infer_datetime_format=True)

    goatdfarr=[]
    for ctype in sorted(goatdf.cardtype.unique()):
        goatdfarr.append(goatdf[goatdf['cardtype']==ctype])
        goatdfarr[ctype]=goatdfarr[ctype].drop(columns='cardtype')
        goatdfarr[ctype].columns=['price'+str(ctype)]

    cur.execute('''SELECT carddate,AVG(pshipq) as AVGE,percentile_cont(0.5) WITHIN GROUP (ORDER BY pshipq) as MED
        FROM (SELECT DISTINCT title,cardname,carddate,(price+shipping)/COALESCE(availablequant,cardquantity,maybecardquantity,1) as pshipq FROM public.transactions
        WHERE possiblybad IS FALSE
        AND ismisprint IS NULL
        AND cardname=%s
        AND isfoil IS FALSE
        AND (cardlanguage='english' or cardlanguage IS NULL)
        AND (cardset IS NULL OR cardset NOT IN('LEA','LEB','U'))
        AND (isemblem IS NULL OR isemblem IS FALSE)
        AND (lotis IS FALSE OR lotis IS NULL)
        AND (isboosterbox IS NULL OR isboosterbox IS FALSE)
        AND cardspecial IS NULL
        AND issleeve IS NULL
        AND isdeck IS NULL
        AND isplaymat IS NULL
        AND isultra IS NULL
        AND (istoken IS NULL OR istoken IS FALSE)
        AND (iscommanderdeck IS NULL OR iscommanderdeck IS FALSE)
        AND isgraded IS NULL
        AND (othercards IS NULL OR transplit IS TRUE)
        AND saletype IN('normal')
        AND price+shipping<100) as foo WHERE pshipq<50 
        GROUP BY carddate ORDER BY carddate ASC''',analyname)

    ebayrows=cur.fetchall()

    #ebaydf=pd.DataFrame(ebayrows,columns=["carddate","avg","med"]) 
    ebaydf=pd.DataFrame(ebayrows,columns=["carddate","AVG","MED"]).set_index("carddate")
    ebaydf['MINAVGMED']=ebaydf.apply(lambda row: np.minimum(row.AVG,row.MED), axis = 1)
    ebaydf=ebaydf.drop(columns=['AVG','MED'])
    ebaydf.index=pd.to_datetime(ebaydf.index,infer_datetime_format=True)

    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.tourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.tourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    piotournrows=cur.fetchall()

    piotourndf=pd.DataFrame(piotournrows,columns=["carddate","pioratio"]).set_index("carddate")
    piotourndf=piotourndf.astype({'pioratio': 'float64'})
    
    piotourndf.index=pd.to_datetime(piotourndf.index,infer_datetime_format=True)

    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.moderntourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.moderntourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    moderntournrows=cur.fetchall()

    moderntourndf=pd.DataFrame(moderntournrows,columns=["carddate","modratio"]).set_index("carddate")
    moderntourndf=moderntourndf.astype({'modratio': 'float64'})
    
    moderntourndf.index=pd.to_datetime(moderntourndf.index,infer_datetime_format=True)
        
    missingdatefix=pd.date_range(start='2019-10-15',end=end_dataset)
    
    moderntourndf=moderntourndf.reindex(missingdatefix).fillna("NaN")
    
    moderntourndf.index.name='carddate'
    

    cur.execute('''SELECT entrydate1 as entrydate,SUM(cardquant)/AVG(aggcount) as ratio FROM(
    SELECT DISTINCT cardname0,entrydate1,cardquant,aggcount,tourntitle FROM
    (SELECT cardname as cardname0,entrydate as entrydate0,cardquant,tourntitle FROM public.standardtourninfo WHERE cardname ILIKE %s) AS foo1,
    (SELECT entrydate as entrydate1,SUM(cardquant) as aggcount FROM public.standardtourninfo GROUP BY entrydate) AS bar
    WHERE entrydate0=entrydate1) AS foo
    GROUP BY cardname0,entrydate1''',analyname)

    standardtournrows=cur.fetchall()
    
    if len(standardtournrows)==0:
        standardtourndf=pd.DataFrame([['2020-04-03',0],['2020-04-04',0]],columns=["carddate","standratio"]).set_index("carddate")
    else:
        standardtourndf=pd.DataFrame(standardtournrows,columns=["carddate","standratio"]).set_index("carddate")
        
    standardtourndf=standardtourndf.astype({'standratio': 'float64'})
    standardtourndf.index=pd.to_datetime(standardtourndf.index,infer_datetime_format=True)

    goatdfarr.insert(0,ebaydf)
    goatdfarr.append(piotourndf)
    goatdfarr.append(moderntourndf)
    goatdfarr.append(standardtourndf)
    overalldf=pd.concat(goatdfarr,axis=1,sort=True)

#     overalldf.index=pd.to_datetime(overalldf.index,infer_datetime_format=True)

#     overalldf.index.name='carddate'
    
    overalldf=pd.concat([overalldf['MINAVGMED'].fillna("NaN"),
                         overalldf.loc[:, overalldf.columns.difference(['MINAVGMED'])].fillna(method='ffill').fillna(method='bfill')],
                        axis=1,join='inner')
    



    dynamic_feat_arr=[]
    for col in overalldf.columns[1:]:
        dynamic_feat_arr.append(list(overalldf[start_date-timedelta(prediction_length):end_training-timedelta(prediction_length)][col]))



    dynamic_feat_arr_test=[]

    for col in overalldf.columns[1:]:
        dynamic_feat_arr_test.append(list(overalldf[start_date-timedelta(days=prediction_length):end_dataset-timedelta(days=prediction_length)][col]))
    
    if(np.array(dynamic_feat_arr).shape[1]==len(overalldf[start_date:end_training]['MINAVGMED']) and
      np.array(dynamic_feat_arr_test).shape[1]==len(overalldf[start_date:end_dataset]['MINAVGMED'])):
        training_data.append({"start":str(start_date),
                       "target":list(overalldf[start_date:end_training]['MINAVGMED']),
                        "cat":[cat],
                       "dynamic_feat":dynamic_feat_arr})


        test_data.append({"start":str(start_date),
                       "target":list(overalldf[start_date:end_dataset]['MINAVGMED']),
                          "cat":[cat],
                       "dynamic_feat":dynamic_feat_arr_test})
        
        target_series.append(overalldf[start_date:end_dataset]['MINAVGMED'])
    else:
        print(str(analyname)+str('error'))
        sys.exit()
#for loop over cardnames ends here

('Marauding Raptor',)
('Desecrated Tomb',)
('Dragonskull Summit',)
('Tocatli Honor Guard',)
('Sphinx of Foresight',)
('Dragonlord Dromoka',)
('Hour of Revelation',)
('Llanowar Wastes',)
('Risk Factor',)
('Linvala, the Preserver',)
('Needle Spires',)
('Steel Leaf Champion',)
('Rekindling Phoenix',)
('Fae of Wishes',)
('Sunpetal Grove',)
('Hagra Mauling',)
('Narcomoeba',)
('Zurgo Bellstriker',)
('Escape to the Wilds',)
('Klothys, God of Destiny',)
('Kozilek, the Great Distortion',)
('Phyrexian Revoker',)
('Idyllic Tutor',)
('Eldritch Evolution',)
('Steam Vents',)
("Sigarda's Aid",)
('Terror of the Peaks',)
('Gideon of the Trials',)
('Valakut Awakening',)
('Elvish Clancaller',)
('Omnath, Locus of Creation',)
('Gideon, Ally of Zendikar',)
('Icon of Ancestry',)
("Vivien's Arkbow",)
('Finale of Devastation',)
('Mausoleum Wanderer',)
('Growth-Chamber Guardian',)
('Karn, the Great Creator',)
('Rotting Regisaur',)
('Lazav, the Multifarious',)
('Blast Zone',)
('Confounding Conundrum',)
('Castle 

In [ ]:
cur.close()
conn.close()

In [ ]:
overalldf['MINAVGMED']

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode('utf-8'))


write_dicts_to_file("train.json", training_data)
write_dicts_to_file("test.json",test_data)

In [ ]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)


In [ ]:
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

In [ ]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

In [ ]:
overalldf

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    base_job_name='deepar-mtgml',
    output_path=s3_output_path
)

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length)
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels, wait=True)

In [ ]:
class DeepARPredictor(sagemaker.predictor.RealTimePredictor):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type=sagemaker.content_types.CONTENT_TYPE_JSON, **kwargs)
        
    def predict(self, ts, cat=None, dynamic_feat=None, 
                num_samples=100, return_samples=False, quantiles=["0.1", "0.5", "0.9"]):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.
        
        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])
        
        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + timedelta(1)
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)
    
    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None)

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')
    
    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode('utf-8'))['predictions'][0]
        prediction_length = len(next(iter(predictions['quantiles'].values())))
        prediction_index = pd.DatetimeIndex(start=prediction_time, freq=freq, periods=prediction_length)        
        if return_samples:
            dict_of_samples = {'sample_' + str(i): s for i, s in enumerate(predictions['samples'])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(data={**predictions['quantiles'], **dict_of_samples}, index=prediction_index)

    def set_frequency(self, freq):
        self.freq = freq
        
def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]        

def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat        
    return obj

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    predictor_cls=DeepARPredictor)

In [ ]:
dynamic_feat_pred=[]
for col in overalldf.columns[1:]:
    dynamic_feat_pred.append(list(overalldf[start_date-timedelta(prediction_length):end_dataset][col]))

In [ ]:
predictor.predict(ts=overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').asfreq(freq),dynamic_feat=dynamic_feat_pred, quantiles=[0.10, 0.5, 0.90])["0.5"]
#.resample('D').mean()

In [ ]:
def plot(
    predictor, 
    target_ts, 
    cat=None, 
    dynamic_feat=None, 
    forecast_date=end_training, 
    show_samples=False, 
    plot_history=7,
    confidence=80
):
    print("calling served model to generate predictions starting from {}".format(str(forecast_date)))
    assert(confidence > 50 and confidence < 100)
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5
        
    # we first construct the argument to call our model
    args = {
        "ts": target_ts[:forecast_date],
        "return_samples": show_samples,
        "dynamic_feat":dynamic_feat,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100
    }


    if dynamic_feat is not None:
        args["dynamic_feat"] = dynamic_feat
        fig = plt.figure(figsize=(20, 6))
        ax = plt.subplot(2, 1, 1)
    else:
        fig = plt.figure(figsize=(20, 3))
        ax = plt.subplot(1,1,1)
    
    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, 'cat = {}'.format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples: 
        for key in prediction.keys():
            if "sample" in key:
                #prediction[key].plot(color='lightskyblue', alpha=0.2, label='_nolegend_')
                plt.plot(prediction[key])
                
                
    # plot the target
    target_section = target_ts[forecast_date-plot_history:forecast_date+prediction_length]
    #print(target_section)
    target_section=target_section.interpolate(method='linear')
    #print(target_section)
    #target_section.plot()
    plt.plot(target_section)

    
    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index, 
        prediction[str(low_quantile)].values, 
        prediction[str(up_quantile)].values, 
        color="b", alpha=0.3, label='{}% confidence interval'.format(confidence)
    )
    print(prediction["0.5"])
    #prediction["0.5"].plot(color="b", label='P50').scatter()
    plt.plot(prediction["0.5"])
    ax.legend(loc=2)    
    
    # fix the scale as the samples may change it
    ax.set_ylim(target_section.min() * 0.5, target_section.max() * 1.5)
    
    if dynamic_feat is not None:
        for i, f in enumerate(dynamic_feat, start=1):
            ax = plt.subplot(len(dynamic_feat) * 2, 1, len(dynamic_feat) + i, sharex=ax)
            feat_ts = pd.Series(
                index=pd.DatetimeIndex(start=target_ts.index[0], freq=target_ts.index.freq, periods=len(f)),
                data=f
            )
            #feat_ts[forecast_date-plot_history:forecast_date+prediction_length].plot(ax=ax, color='g')
            plt.plot(feat_ts[forecast_date-plot_history:forecast_date+prediction_length])



In [ ]:
style = {'description_width': 'initial'}

In [ ]:
@interact_manual(
    card_id=IntSlider(min=0, max=701, value=91, style=style), ###NEED TO DO SOMETHING WITH THIS
    forecast_day=IntSlider(min=0, max=365, value=0, style=style),
    confidence=IntSlider(min=60, max=95, value=80, step=5, style=style),
    history_weeks_plot=IntSlider(min=0, max=10, value=1, style=style),
    show_samples=Checkbox(value=False),
    continuous_update=False
)
def plot_interact( forecast_day, confidence, history_weeks_plot, show_samples):
    plot(
        predictor,
        dynamic_feat=dynamic_feat_pred,
        target_ts=overalldf[start_date:end_dataset]['MINAVGMED'].astype('float64').asfreq(freq),
        #.resample('D').mean(),
        forecast_date=end_training + datetime.timedelta(days=forecast_day),
        show_samples=show_samples,
        plot_history=history_weeks_plot*7,
        confidence=confidence
    )


In [ ]:
end_training